In [1]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
from matplotlib import pyplot

In [2]:
# Load and prepare MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train) , (x_test, y_test) = mnist.load_data()
# Normalize dataset # (rgb is from 0 to 255)
x_train = x_train / 255.0
x_test = x_test / 255.0

# transform data shapes
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [3]:
# show few values from the dataset
def showData(x_train, x_test, y_train, y_test):
    print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
    print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))
    # plot first few images
    for i in range(9):
        # define subplots position
        pyplot.subplot(330 + 1 + i)
        # plot raw pixel data
        pyplot.imshow(x_test[i], cmap=pyplot.get_cmap('gray'))
    # show the figure
    pyplot.show()
# showData(x_train, x_test, y_train, y_test)

In [4]:
def prepareModel():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(3, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(80, activation='elu'))
    model.add(tf.keras.layers.Dense(60, activation='elu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(10))
    return model

In [5]:
def compileAndFitModel(model, epochs=5):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    learning_rate = 1e-3
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=32)
    return model

In [6]:
tfLiteModelName = 'tfliteModel.tflite'
def saveTFModel(model):
    # convert the model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # save the model
    with open(tfLiteModelName, 'wb') as f:
        f.write(tflite_model)

In [7]:
# load images to features and labels
def load_images_to_data(x_data, y_data):
    list_of_files = os.listdir("MyNumbers")
    for file in list_of_files:
        image_file_name = os.path.join("MyNumbers", file)
        if ".png" in image_file_name:
            img_o = Image.open(image_file_name).convert("L")
            # img = np.resize(img_o, (28,28))
            img = img_o.resize((28,28))

            im2arr = np.array(img)
            # im2arr = im2arr.reshape(1,28,28)
            im2arr = im2arr.reshape(1,28,28,1)
            
            # pyplot.subplot(330 + 1)
            # pyplot.imshow(img_o, cmap=pyplot.get_cmap('gray'))
            # pyplot.show()
            
            if len(x_data) == 0:
                x_data = im2arr
            else:
                x_data = np.append(x_data, im2arr, axis=0)
            
            if len(y_data) == 0:
                y_data = [np.uint8(file[0])]
            else:
                y_data = np.append(y_data, [np.uint8(file[0])], axis=0)
    return x_data, y_data

def evaluateModelOnMyData(model, show_data = False):
    x_test_my = []
    y_test_my = []
    x_test_my, y_test_my = load_images_to_data(x_test_my, y_test_my)

    x_test_my = x_test_my / 255.0
    
    print(y_test_my)
    
    if show_data:
        showData(x_train, x_test_my, y_train, y_test_my)
    
    # evaluate my final model's performance on my handwritten numbers
    model.evaluate(x_test_my, y_test_my);

In [8]:
model = prepareModel()
model = compileAndFitModel(model, epochs=100)
# # Evaluate model performance
model.evaluate(x_test, y_test, verbose=2)
saveTFModel(model)
print('Evaluation on my handwritten digits')
evaluateModelOnMyData(model)#, True)

Epoch 1/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3113 - accuracy: 0.9044
Epoch 2/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1549 - accuracy: 0.9526
Epoch 3/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1154 - accuracy: 0.9642
Epoch 4/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0897 - accuracy: 0.9720
Epoch 5/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0730 - accuracy: 0.9774
Epoch 6/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0608 - accuracy: 0.9809
Epoch 7/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0497 - accuracy: 0.9836
Epoch 8/100
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0443 - accuracy: 0.9857
Epoch 9/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0379 - accuracy: 0.9877
Epoch 10/100
1875/1875 [==============================] - 12s 6m